<a href="https://colab.research.google.com/github/Navya-20JR1A0568/20JR1A0568/blob/main/ML_Sentimental_Analysis_week_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.4.1
!pip install spark-nlp==5.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=39d2b80ed1f67ff23d5500d70421af3eddbc55d7ad41163bfeaf288cea8ef70d
  Stored in directory: /root/.cache/pip/wheels/e9/b4/d8/38accc42606f6675165423e9f0236f8e825f6b6b6048d6743e
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires pyspark[connect]~=3.5.1, but you have pyspark 3.4.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.6/547.6 kB 1

PySpark is installed in order to process large amounts of data.Installs Spark NLP to process natural language in Spark.Versions have been fixed to ensure compatibility.The! allows us to execute shell commands on a notebook.The remainder of the code wouldn't function without these.


In [2]:
from pyspark.sql import SparkSession
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import Tokenizer, Normalizer, BertEmbeddings, SentimentDLModel, LemmatizerModel, UniversalSentenceEncoder
from pyspark.ml import Pipeline as MLPipeline


import sparknlp

spark = sparknlp.start()

For distributed computing, import Spark.To handle data in a table format, import Pandas.For AI models, import the Hugging Face pipeline.To evaluate, import scikit-learn metrics.For cleaner notebook output, hide warnings.

In [3]:
sentences = [
    "I love this movie!", "This was the worst experience.", "Pretty decent overall.",
    "Absolutely fantastic!", "I'm not sure how I feel.", "Worst purchase ever.",
    "Great value for the money.", "It was okay, not great.", "Terrible, just terrible.",
    "Super fun and engaging!", "Would not recommend it.", "Kind of boring.",
    "Loved every minute!", "It was a disaster.", "Highly recommend!",
    "Too expensive for what you get.", "Amazing support team!", "Horrible food.",
    "I'll definitely buy it again.", "Meh, nothing special.", "Exceeded my expectations!",
    "Not worth the hype.", "Incredible storytelling.", "Never again.",
    "Pretty enjoyable!", "Worst customer service.", "Delightful and fresh.",
    "Disappointed.", "Can't wait to try it again!", "A total waste of time.",
    "Superb work!", "Very underwhelming.", "Loved the packaging.",
    "Felt very rushed.", "Unbelievably good.", "Didn't like it at all.",
    "Perfect execution!", "Mediocre at best.", "Simply amazing.",
    "It made my day!", "Forgettable.", "Super smooth experience.",
    "I regret buying it.", "Truly inspiring.", "Boring and repetitive.",
    "Five stars!", "Nothing new.", "Highly entertaining.", "Wasted potential.", "It was alright."
]

Create a Python dictionary using labels and sentences as keys.Sentences are brief passages used to gauge sentiment.Positive, negative, and neutral labels are the appropriate responses.This serves as our "ground truth" when we test the model.We'll compare predictions to these labels later.


In [4]:
df = spark.createDataFrame([(s,) for s in sentences], ["text"])


In [ ]:
Launch a fresh Spark session to begin tasks.Create a Pandas DataFrame from the Python dictionary.Create a Spark DataFrame from a Pandas DataFrame.Return to Pandas so that Hugging Face can utilize it.
This makes it simple to transfer data between Spark and Pandas.

In [5]:
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")


In [6]:
use = UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [7]:
sentiment_model = SentimentDLModel.pretrained("sentimentdl_use_twitter", "en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


In [11]:
pipeline = Pipeline(stages=[document_assembler, use, sentiment_model])


In [12]:
result = pipeline.fit(df).transform(df)


In [13]:
result.select("text", "sentiment.result").show(truncate=False)

+-------------------------------+----------+
|text                           |result    |
+-------------------------------+----------+
|I love this movie!             |[positive]|
|This was the worst experience. |[negative]|
|Pretty decent overall.         |[positive]|
|Absolutely fantastic!          |[positive]|
|I'm not sure how I feel.       |[negative]|
|Worst purchase ever.           |[negative]|
|Great value for the money.     |[positive]|
|It was okay, not great.        |[neutral] |
|Terrible, just terrible.       |[negative]|
|Super fun and engaging!        |[positive]|
|Would not recommend it.        |[negative]|
|Kind of boring.                |[negative]|
|Loved every minute!            |[positive]|
|It was a disaster.             |[negative]|
|Highly recommend!              |[positive]|
|Too expensive for what you get.|[negative]|
|Amazing support team!          |[positive]|
|Horrible food.                 |[negative]|
|I'll definitely buy it again.  |[positive]|
|Meh, noth

In [14]:

from pyspark.sql import SparkSession
import pandas as pd
from transformers import pipeline
from sklearn.metrics import classification_report, accuracy_score
import warnings


warnings.filterwarnings("ignore")


data = {
    "sentence": [
        "I love this product!", "This is the worst service ever.", "Absolutely fantastic experience.",
        "I'm not happy with the results.", "The movie was okay, not great.", "What a wonderful surprise!",
        "I would not recommend this.", "Such a delightful day.", "Terrible customer support.",
        "This phone is amazing!", "Very disappointing performance.", "I'm so excited about this!",
        "Could be better.", "Totally satisfied with my purchase.", "I hate how this works.",
        "It exceeded my expectations!", "Nothing special about it.", "I'm impressed by the quality.",
        "Worst purchase I've made.", "A pretty decent option."
    ],
    "label": [
        "positive", "negative", "positive", "negative", "neutral", "positive", "negative",
        "positive", "negative", "positive", "negative", "positive", "neutral", "positive",
        "negative", "positive", "neutral", "positive", "negative", "neutral"
    ]
}

In [15]:
spark = SparkSession.builder.appName("LLM Sentiment Evaluation").getOrCreate()


In [16]:
df_pd = pd.DataFrame(data)
df_spark = spark.createDataFrame(df_pd)


In [17]:
df = df_spark.toPandas()

In [18]:
classifier = pipeline("sentiment-analysis")  # Defaults to distilbert-base-uncased-finetuned-sst-2-english


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


build a pipeline for sentiment analysis using Hugging Face.It loads a DistilBERT model by default.Either positive or negative labels are predicted by this model.Short sentences are a good fit for it.It will be used to categorize our dataset.

In [19]:
def map_prediction(pred):
    label = pred['label'].lower()
    if label == 'positive':
        return 'positive'
    elif label == 'negative':
        return 'negative'
    else:
        return 'neutral'

df['predicted'] = df['sentence'].apply(lambda x: map_prediction(classifier(x)[0]))


Create a function that will standardize model labels.Make the output lowercase.Keep things both positive and negative.As a backup, add neutral.Use this in each sentence and keep track of your predictions.

In [20]:
print("\nClassification Report:")
print(classification_report(df['label'], df['predicted'], digits=3))

print("\nAccuracy Score:", accuracy_score(df['label'], df['predicted']))



Classification Report:
              precision    recall  f1-score   support

    negative      0.700     1.000     0.824         7
     neutral      0.000     0.000     0.000         4
    positive      0.900     1.000     0.947         9

    accuracy                          0.800        20
   macro avg      0.533     0.667     0.590        20
weighted avg      0.650     0.800     0.715        20


Accuracy Score: 0.8


Compare the actual labels with the model's predictions.Display each label's F1 score, precision, and recall.For tidy results, use three decimal places.Print overall accuracy as well.This indicates how well the model performed.

In [21]:
df_result_spark = spark.createDataFrame(df)
df_result_spark.show(truncate=False)


spark.stop()


+-----------------------------------+--------+---------+
|sentence                           |label   |predicted|
+-----------------------------------+--------+---------+
|I love this product!               |positive|positive |
|This is the worst service ever.    |negative|negative |
|Absolutely fantastic experience.   |positive|positive |
|I'm not happy with the results.    |negative|negative |
|The movie was okay, not great.     |neutral |negative |
|What a wonderful surprise!         |positive|positive |
|I would not recommend this.        |negative|negative |
|Such a delightful day.             |positive|positive |
|Terrible customer support.         |negative|negative |
|This phone is amazing!             |positive|positive |
|Very disappointing performance.    |negative|negative |
|I'm so excited about this!         |positive|positive |
|Could be better.                   |neutral |negative |
|Totally satisfied with my purchase.|positive|positive |
|I hate how this works.        

Create a Spark DataFrame from the final Pandas DataFrame (with predictions).Working with big data is made simpler as a result.To see the results, use.show().truncate=False preserves the entire text.This step is for workflows that use Spark.

In [22]:
from transformers import pipeline
generator = pipeline("text-generation", model="gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Bring in the pipeline for Hugging Face.To generate text, load GPT-2.Start it off with a prompt.Request that it produce fifty tokens.Print the continuation produced by AI.

In [23]:
prompt = "Artificial intelligence is transforming"

In [24]:
output = generator(prompt, max_length=50, num_return_sequences=1)
print("Step 2:Generated Text:\n", output[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Step 2:Generated Text:
 Artificial intelligence is transforming our lives, and it will take us some time before we're able to get to grips with how we're doing it. But there's no reason to take that away from your kids and your friends, because we've also learned that we have the ability to make a life for ourselves and our kids.

We've learned that having kids is a very important part of our family. I know I have. There's a lot of kids who are not having kids, or are struggling to get by, that are struggling to make ends meet. So, I think we've learned that our kids have a way of life that we don't know, that we can be very proud of. And for me personally, I think that's something that we've seen more and more, that we can be proud of. I think that's really important.

I think that's a big part of what I like about the human condition. I think that my kids are having a kind of life that we don't know about. And so, I know that because I think that we've learned that our children are h

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")


Load the model and tokenizer for GPT-2.Tokenizer converts text to tokens, which are numbers.For PyTorch format, use return_tensors="pt".Token IDs should be printed for the prompt.GPT-2 recognizes these IDs.

In [ ]:
tokens = tokenizer(prompt, return_tensors="pt")
print("\nToken IDs:\n", tokens['input_ids'][0].tolist())


Token IDs:
 [8001, 9542, 4430, 318, 25449]


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")

In [ ]:
tokens = tokenizer(prompt, return_tensors="pt")
print("\nToken IDs:\n", tokens['input_ids'][0].tolist())


Token IDs:
 [8001, 9542, 4430, 318, 25449]


In [ ]:
with torch.no_grad():
    embeddings = model(**tokens).last_hidden_state
print("\nEmbeddings Shape:\n", embeddings.shape)


Embeddings Shape:
 torch.Size([1, 5, 768])


Run tokens through GPT-2 without using the training mode.Obtain the word embeddings for the last hidden state.These are vectors of numbers that convey meaning.Shape displays features, batch, and sequence length.beneficial for similarity or semantic analysis.

In [ ]:
from IPython.display import Markdown
Markdown("""
### 🧠 Discussion Points

- **Tokenization**: Converts text into subword units and token IDs.
- **Embeddings**: Token IDs are mapped to dense vectors capturing meaning.
- **Transformer Layers**: Use attention to understand relationships between tokens.
- **Text Generation**: Predicts next words based on context and learned patterns.
""")


### 🧠 Discussion Points

- **Tokenization**: Converts text into subword units and token IDs.
- **Embeddings**: Token IDs are mapped to dense vectors capturing meaning.
- **Transformer Layers**: Use attention to understand relationships between tokens.
- **Text Generation**: Predicts next words based on context and learned patterns.


Bring in a Markdown notebook display.Use ### for headings when writing formatted notes.For a concise explanation, use bullet points.Put it in the notebook.This gives the explanation a polished appearance.

In [ ]:
!pip install transformers torch scikit-learn

Put the Hugging Face Transformers in place.For deep learning, install PyTorch.Install scikit-learn to access assessment tools.Here, versions are not fixed.required for the notebook's later sections

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Select a tweet-trained RoBERTa model.For text preprocessing, load its tokenizer.
Load the sentiment classification model.Use both to build a pipeline.Positive, negative, and neutral signals can all be detected by this model.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load RoBERTa model fine-tuned for sentiment analysis
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


Device set to use cpu


In [ ]:
test_sentences = [
    "I absolutely love this!",
    "This is so frustrating and annoying.",
    "What a beautiful day!",
    "I can't stand this anymore.",
    "Totally worth it!",
    "Worst experience ever.",
    "I'm really happy with the results.",
    "This is not what I expected.",
    "Amazing job!",
    "Terrible service."
]


true_labels = [
    "positive", "negative", "positive", "negative", "positive",
    "negative", "positive", "negative", "positive", "negative"]


Make a test sentence list.Give each the appropriate label.Labels correspond to the output types of the model.This serves as the ground truth.It will be used to verify the accuracy of the model

In [ ]:
label_mapping = {'label_0': 'negative', 'label_1': 'neutral', 'label_2': 'positive'}
predicted_labels = [label_mapping[sentiment_pipeline(text)[0]['label'].lower()] for text in test_sentences]
print(predicted_labels)

['positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative']


Translate the numerical labels of RoBERTa into words.Apply the model to every test sentence.Obtain the anticipated label from the outcome.Change it to lowercase.Keep a list of all the predictions.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate using multiclass metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

print("Evaluation Metrics:")
print(f"Accuracy:  {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1 Score:  {f1:.2f}")

Evaluation Metrics:
Accuracy:  1.00
Precision: 1.00
Recall:    1.00
F1 Score:  1.00


Determine the overall accuracy.Determine precision equally for every class.Recall should be calculated equally for each class.Determine the balanced measure's F1 score.These metrics demonstrate the quality of the model.


In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from transformers import pipeline
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings("ignore")

Launch Spark and import the UDF tools.Create a function that loads the model only once.Create a sentiment prediction function.It should be registered as a Spark UDF.Show the results after applying it to the DataFrame.

In [ ]:
data = {
    "sentence": [
        "I love this product!", "This is the worst service ever.", "Absolutely fantastic experience.",
        "I'm not happy with the results.", "The movie was okay, not great.", "What a wonderful surprise!",
        "I would not recommend this.", "Such a delightful day.", "Terrible customer support.",
        "This phone is amazing!", "Very disappointing performance.", "I'm so excited about this!",
        "Could be better.", "Totally satisfied with my purchase.", "I hate how this works.",
        "It exceeded my expectations!", "Nothing special about it.", "I'm impressed by the quality.",
        "Worst purchase I've made.", "A pretty decent option."
    ],
    "label": [
        "positive", "negative", "positive", "negative", "neutral", "positive", "negative",
        "positive", "negative", "positive", "negative", "positive", "neutral", "positive",
        "negative", "positive", "neutral", "positive", "negative", "neutral"
    ]
}

In [ ]:
spark = SparkSession.builder.appName("LLM Sentiment Evaluation").getOrCreate()


In [ ]:
df_pd = pd.DataFrame(data)
df_spark = spark.createDataFrame(df_pd)

In [ ]:
df = df_spark.toPandas()

In [ ]:

classifier = pipeline("sentiment-analysis")  # Defaults to distilbert-base-uncased-finetuned-sst-2-english


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [ ]:
def map_prediction(pred):
    label = pred['label'].lower()
    if label == 'positive':
        return 'positive'
    elif label == 'negative':
        return 'negative'
    else:
        return 'neutral'

df['predicted'] = df['sentence'].apply(lambda x: map_prediction(classifier(x)[0]))


In [ ]:
print("\nClassification Report:")
print(classification_report(df['label'], df['predicted'], digits=3))

print("\nAccuracy Score:", accuracy_score(df['label'], df['predicted']))



Classification Report:
              precision    recall  f1-score   support

    negative      0.700     1.000     0.824         7
     neutral      0.000     0.000     0.000         4
    positive      0.900     1.000     0.947         9

    accuracy                          0.800        20
   macro avg      0.533     0.667     0.590        20
weighted avg      0.650     0.800     0.715        20


Accuracy Score: 0.8


In [ ]:
df_result_spark = spark.createDataFrame(df)
df_result_spark.show(truncate=False)


+-----------------------------------+--------+---------+
|sentence                           |label   |predicted|
+-----------------------------------+--------+---------+
|I love this product!               |positive|positive |
|This is the worst service ever.    |negative|negative |
|Absolutely fantastic experience.   |positive|positive |
|I'm not happy with the results.    |negative|negative |
|The movie was okay, not great.     |neutral |negative |
|What a wonderful surprise!         |positive|positive |
|I would not recommend this.        |negative|negative |
|Such a delightful day.             |positive|positive |
|Terrible customer support.         |negative|negative |
|This phone is amazing!             |positive|positive |
|Very disappointing performance.    |negative|negative |
|I'm so excited about this!         |positive|positive |
|Could be better.                   |neutral |negative |
|Totally satisfied with my purchase.|positive|positive |
|I hate how this works.        

In [ ]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from transformers import pipeline


In [ ]:
spark = SparkSession.builder.appName("SparkLLMSentiment").getOrCreate()

In [ ]:
data = [
    ("I love this product!",),
    ("This is the worst service ever.",),
    ("Absolutely fantastic experience.",),
    ("I'm not happy with the results.",),
    ("The movie was okay, not great.",),
    ("What a wonderful surprise!",),
    ("I would not recommend this.",),
    ("Such a delightful day.",),
    ("Terrible customer support.",),
    ("This phone is amazing!",),
    ("Very disappointing performance.",),
    ("I'm so excited about this!",),
    ("Could be better.",),
    ("Totally satisfied with my purchase.",),
    ("I hate how this works.",),
    ("It exceeded my expectations!",),
    ("Nothing special about it.",),
    ("I'm impressed by the quality.",),
    ("Worst purchase I've made.",),
    ("A pretty decent option.",)
]

columns = ["sentence"]
df = spark.createDataFrame(data, columns)

In [ ]:
def load_model():
    return pipeline("sentiment-analysis")

def predict_sentiment(text):
    global clf
    if "clf" not in globals():
        clf = load_model()
    result = clf(text)[0]['label'].lower()
    return result

In [ ]:
sentiment_udf = udf(predict_sentiment, StringType())
df_with_predictions = df.withColumn("predicted_sentiment", sentiment_udf("sentence"))
df_with_predictions.show(truncate=False)

+-----------------------------------+-------------------+
|sentence                           |predicted_sentiment|
+-----------------------------------+-------------------+
|I love this product!               |positive           |
|This is the worst service ever.    |negative           |
|Absolutely fantastic experience.   |positive           |
|I'm not happy with the results.    |negative           |
|The movie was okay, not great.     |negative           |
|What a wonderful surprise!         |positive           |
|I would not recommend this.        |negative           |
|Such a delightful day.             |positive           |
|Terrible customer support.         |negative           |
|This phone is amazing!             |positive           |
|Very disappointing performance.    |negative           |
|I'm so excited about this!         |positive           |
|Could be better.                   |negative           |
|Totally satisfied with my purchase.|positive           |
|I hate how th

based sentiment analysis in SparkWe import UDF tools and launch Spark.To load the model once, we define a function.To predict sentiment, we define a function.It is registered as a Spark UDF.We use it on a DataFrame and display the outcomes.

In [ ]:
spark.stop()

merely ends the Spark session.After Spark work is finished, it is best practice to terminate all background processes connected to Spark, free up memory and CPU resources, and guarantee that no more Spark code will run.